In [30]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# CHANGING LOCATION MINIMUM REQUIREMENT

# Number of locations and time periods
L = 6              # 6 fishing locations
T = 36             # 36 time periods



# p = np.array([
#     [... 36 values for River          ...],
#     [... 36 values for Cliff Top      ...],
#     [... 36 values for Mouth of River ...],
#     [... 36 values for Pond           ...],
#     [... 36 values for Sea            ...],
#     [... 36 values for Pier           ...]
# ])
p = np.array([
    [380.2352941,402.4390244,402.4390244,380.2352941,402.4390244,402.4390244,369.6296296,387.4285714,387.4285714,312.7272727,280.3636364,451.0714286,421.2698413,436.4705882,638.7755102,737.704918,1489.387755,1378.4,794.6875,1405.090909,1307.5,791.9402985,1531.47541,1433.225806,1210,1953.469388,1837.142857,538.9041096,706.1764706,906.7647059,441.8181818,548.3783784,549.7297297,388.1395349,405.1764706,405.1764706],
    [10,2151.428571,2151.428571,10,2151.428571,2151.428571,10,2553.75,2553.75,10,1678.75,1678.75,10,1775.714286,1775.714286,10,775.7142857,775.7142857,10,10,10,10,10,10,969.0909091,1775.714286,1775.714286,1292.222222,1678.75,1678.75,969.0909091,1638.823529,1638.823529,10,2151.428571,2151.428571],
    [2507.5,2507.5,2507.5,2507.5,2507.5,2507.5,1437.142857,1437.142857,1437.142857,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1033.125,1033.125,1033.125,1437.142857,1437.142857,1437.142857,4006,4006,4006,1437.142857,1437.142857,1437.142857],
    [640,936.1904762,936.1904762,640,936.1904762,936.1904762,519.1666667,754.4,706.4,401.5,520.9756098,526.5,342.7906977,387.6595745,459.5918367,572,574.2857143,706.25,600.4761905,571.372549,694.8,644.1025641,633.7777778,777.2727273,597.6470588,811.3043478,1037.391304,833,868.8888889,868.8888889,833,750.5882353,750.5882353,913.6842105,1145.263158,1145.263158],
    [986.4583333,1088.020833,1492.1875,989.0625,1090.625,1492.1875,1175.520833,1285.9375,1441.666667,994.7916667,994.7916667,1154.6875,1194.387755,1194.387755,1376.020408,1137.244898,1596.938776,1762.244898,1195.3125,1673.958333,1720.3125,1254.6875,1747.938144,1710.309278,1268.229167,1753.092784,1777.319588,1147.44898,1147.44898,1298.979592,1120.833333,1224.479167,1598.958333,986.4583333,1088.020833,1492.1875],
    [3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,3394.444444,3394.444444,3394.444444,3394.444444,2568.75,2568.75,3394.444444,2568.75,2568.75,1507.142857,1507.142857,1507.142857,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333]
])


MAX_CASTS = 30
MIN_VISITS = 5
M = 30

location_names = [
    "River", "Cliff Top", "River Mouth", "Pond", "Sea", "Pier"
]
time_names = ["Jan 9a-4p", "Jan 4a-9a/4p-9p", "Jan 9p-4a",
              "Feb 9a-4p", "Feb 4a-9a/4p-9p", "Feb 9p-4a",
              "Mar 9a-4p", "Mar 4a-9a/4p-9p", "Mar 9p-4a",
              "Apr 9a-4p", "Apr 4a-9a/4p-9p", "Apr 9p-4a",
              "May 9a-4p", "May 4a-9a/4p-9p", "May 9p-4a",
              "Jun 9a-4p", "Jun 4a-9a/4p-9p", "Jun 9p-4a",
              "Jul 9a-4p", "Jul 4a-9a/4p-9p", "Jul 9p-4a",
              "Aug 9a-4p", "Aug 4a-9a/4p-9p", "Aug 9p-4a",
              "Sep 9a-4p", "Sep 4a-9a/4p-9p", "Sep 9p-4a",
              "Oct 9a-4p", "Oct 4a-9a/4p-9p", "Oct 9p-4a",
              "Nov 9a-4p", "Nov 4a-9a/4p-9p", "Nov 9p-4a",
              "Dec 9a-4p", "Dec 4a-9a/4p-9p", "Dec 9p-4a"]

# Loop through number of required locations
for required_locations in range(1, L+1):
    print(f"\n--- Optimal solution for requiring {required_locations} location(s) ---")
    
    model = gp.Model(f"Fishing_{required_locations}")
    model.setParam('OutputFlag', 0)
    
    # Variables
    x = model.addVars(L, T, lb=0, vtype=GRB.CONTINUOUS, name="x")
    y = model.addVars(T, vtype=GRB.BINARY, name="y")
    v = model.addVars(L, vtype=GRB.BINARY, name="v")
    
    # Constraints
    model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) <= MAX_CASTS)
    model.addConstr(gp.quicksum(y[t] for t in range(T)) == 1)
    for l in range(L):
        for t in range(T):
            model.addConstr(x[l,t] <= M * y[t])
    # New constraint: exactly 'required_locations' visited
    model.addConstr(gp.quicksum(v[l] for l in range(L)) == required_locations)
    for l in range(L):
        model.addConstr(gp.quicksum(x[l,t] for t in range(T)) >= MIN_VISITS * v[l])
        for t in range(T):
            model.addConstr(x[l,t] <= M * v[l])
    
    # Objective
    model.setObjective(gp.quicksum(p[l,t]*x[l,t] for l in range(L) for t in range(T)), GRB.MAXIMIZE)
    
    # Solve
    model.optimize()
    
    # Print solution
    if model.Status == GRB.OPTIMAL:
        print("Optimal Expected Profit:", model.ObjVal)
        chosen_time_index = max(range(T), key=lambda t: y[t].X)
        print("Chosen Time Period:", time_names[chosen_time_index])
        visited = [l for l in range(L) if v[l].X > 0.5]
        for l in visited:
            print(f"  {location_names[l]}: {x[l, chosen_time_index].X:.0f} casts")


--- Optimal solution for requiring 1 location(s) ---
Optimal Expected Profit: 120180.0
Chosen Time Period: Nov 9a-4p
  River Mouth: 30 casts

--- Optimal solution for requiring 2 location(s) ---
Optimal Expected Profit: 119066.666665
Chosen Time Period: Nov 9a-4p
  River Mouth: 25 casts
  Pier: 5 casts

--- Optimal solution for requiring 3 location(s) ---
Optimal Expected Profit: 107230.78431
Chosen Time Period: Nov 4a-9a/4p-9p
  Cliff Top: 5 casts
  River Mouth: 20 casts
  Pier: 5 casts

--- Optimal solution for requiring 4 location(s) ---
Optimal Expected Profit: 95195.575975
Chosen Time Period: Nov 9p-4a
  Cliff Top: 5 casts
  River Mouth: 15 casts
  Sea: 5 casts
  Pier: 5 casts

--- Optimal solution for requiring 5 location(s) ---
Optimal Expected Profit: 78918.5171515
Chosen Time Period: Nov 9p-4a
  Cliff Top: 5 casts
  River Mouth: 10 casts
  Pond: 5 casts
  Sea: 5 casts
  Pier: 5 casts

--- Optimal solution for requiring 6 location(s) ---
Optimal Expected Profit: 61637.1658
Cho

In [27]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# CHANGING NUM OF CASTS +1

# Number of locations and time periods
L = 6              # 6 fishing locations
T = 36             # 36 time periods



# p = np.array([
#     [... 36 values for River          ...],
#     [... 36 values for Cliff Top      ...],
#     [... 36 values for Mouth of River ...],
#     [... 36 values for Pond           ...],
#     [... 36 values for Sea            ...],
#     [... 36 values for Pier           ...]
# ])
p = np.array([
    [380.2352941,402.4390244,402.4390244,380.2352941,402.4390244,402.4390244,369.6296296,387.4285714,387.4285714,312.7272727,280.3636364,451.0714286,421.2698413,436.4705882,638.7755102,737.704918,1489.387755,1378.4,794.6875,1405.090909,1307.5,791.9402985,1531.47541,1433.225806,1210,1953.469388,1837.142857,538.9041096,706.1764706,906.7647059,441.8181818,548.3783784,549.7297297,388.1395349,405.1764706,405.1764706],
    [10,2151.428571,2151.428571,10,2151.428571,2151.428571,10,2553.75,2553.75,10,1678.75,1678.75,10,1775.714286,1775.714286,10,775.7142857,775.7142857,10,10,10,10,10,10,969.0909091,1775.714286,1775.714286,1292.222222,1678.75,1678.75,969.0909091,1638.823529,1638.823529,10,2151.428571,2151.428571],
    [2507.5,2507.5,2507.5,2507.5,2507.5,2507.5,1437.142857,1437.142857,1437.142857,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1033.125,1033.125,1033.125,1437.142857,1437.142857,1437.142857,4006,4006,4006,1437.142857,1437.142857,1437.142857],
    [640,936.1904762,936.1904762,640,936.1904762,936.1904762,519.1666667,754.4,706.4,401.5,520.9756098,526.5,342.7906977,387.6595745,459.5918367,572,574.2857143,706.25,600.4761905,571.372549,694.8,644.1025641,633.7777778,777.2727273,597.6470588,811.3043478,1037.391304,833,868.8888889,868.8888889,833,750.5882353,750.5882353,913.6842105,1145.263158,1145.263158],
    [986.4583333,1088.020833,1492.1875,989.0625,1090.625,1492.1875,1175.520833,1285.9375,1441.666667,994.7916667,994.7916667,1154.6875,1194.387755,1194.387755,1376.020408,1137.244898,1596.938776,1762.244898,1195.3125,1673.958333,1720.3125,1254.6875,1747.938144,1710.309278,1268.229167,1753.092784,1777.319588,1147.44898,1147.44898,1298.979592,1120.833333,1224.479167,1598.958333,986.4583333,1088.020833,1492.1875],
    [3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,3394.444444,3394.444444,3394.444444,3394.444444,2568.75,2568.75,3394.444444,2568.75,2568.75,1507.142857,1507.142857,1507.142857,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333]
])


# CONSTANTS
MAX_CASTS = 31
MIN_VISITS = 5
M = 30

# create model
model = gp.Model("Increased Cast")
model.setParam('OutputFlag', 0)

# Variables
x = model.addVars(L, T, lb=0, vtype=GRB.CONTINUOUS, name="x")  # casts
y = model.addVars(T, vtype=GRB.BINARY, name="y")                # time period
v = model.addVars(L, vtype=GRB.BINARY, name="v")                # locations

# Constraints
# 1. Max 30 casts
model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) <= MAX_CASTS)

# 2. Exactly one time period
model.addConstr(gp.quicksum(y[t] for t in range(T)) == 1)

# 3. Link x to y
for l in range(L):
    for t in range(T):
        model.addConstr(x[l,t] <= M * y[t])

# 4. Visit at least 3 locations
model.addConstr(gp.quicksum(v[l] for l in range(L)) >= 3)

# 5. Minimum casts if visited
for l in range(L):
    model.addConstr(gp.quicksum(x[l,t] for t in range(T)) >= MIN_VISITS * v[l])

# 6. No casts at unvisited locations
for l in range(L):
    for t in range(T):
        model.addConstr(x[l,t] <= M * v[l])

# Objective: maximize expected profit
model.setObjective(gp.quicksum(p[l,t]*x[l,t] for l in range(L) for t in range(T)), GRB.MAXIMIZE)

# Solve
model.optimize()

# Print solution
def print_solution(model, x, y, v, time_names, location_names):
    if model.Status == GRB.OPTIMAL:
        print("Optimal Expected Profit:", model.ObjVal)

        # chosen time period
        chosen_time_index = max(range(T), key=lambda t: y[t].X)
        chosen_time_text = time_names[chosen_time_index]
        print("\nChosen Time Period:")
        print(f"  {chosen_time_text}")

        # visited locations
        visited_indices = [l for l in range(L) if v[l].X > 0.5]
        visited_locations = [location_names[i] for i in visited_indices]
        max_len = max(len(name) for name in visited_locations)

        print("\nVisited Locations:")
        for l in visited_indices:
            casts = x[l, chosen_time_index].X
            print(f"  {location_names[l]:<{max_len}} : {casts:>3.0f} casts")

location_names = [
    "River", "Cliff Top", "River Mouth", "Pond", "Sea", "Pier"
]


time_names = [
    "Jan 9a-4p", "Jan 4a-9a/4p-9p", "Jan 9p-4a",
    "Feb 9a-4p", "Feb 4a-9a/4p-9p", "Feb 9p-4a",
    "Mar 9a-4p", "Mar 4a-9a/4p-9p", "Mar 9p-4a",
    "Apr 9a-4p", "Apr 4a-9a/4p-9p", "Apr 9p-4a",
    "May 9a-4p", "May 4a-9a/4p-9p", "May 9p-4a",
    "Jun 9a-4p", "Jun 4a-9a/4p-9p", "Jun 9p-4a",
    "Jul 9a-4p", "Jul 4a-9a/4p-9p", "Jul 9p-4a",
    "Aug 9a-4p", "Aug 4a-9a/4p-9p", "Aug 9p-4a",
    "Sep 9a-4p", "Sep 4a-9a/4p-9p", "Sep 9p-4a",
    "Oct 9a-4p", "Oct 4a-9a/4p-9p", "Oct 9p-4a",
    "Nov 9a-4p", "Nov 4a-9a/4p-9p", "Nov 9p-4a",
    "Dec 9a-4p", "Dec 4a-9a/4p-9p", "Dec 9p-4a",
]


print_solution(model, x, y, v, time_names, location_names)

Optimal Expected Profit: 111236.78431

Chosen Time Period:
  Nov 4a-9a/4p-9p

Visited Locations:
  Cliff Top   :   5 casts
  River Mouth :  21 casts
  Pier        :   5 casts


In [28]:

import gurobipy as gp
from gurobipy import GRB
import numpy as np
from collections import namedtuple

# FORCE BY SEASON


# --- MODEL SETUP ---
L = 6 # 6 locations
T = 36  # 36 time periods (12 months * 3 periods)

# CONSTANTS
MAX_CASTS = 30
MIN_VISITS = 5
M = 30

# Expected profit matrix p[l, t]
p = np.array([
    [380.2352941,402.4390244,402.4390244,380.2352941,402.4390244,402.4390244,369.6296296,387.4285714,387.4285714,312.7272727,280.3636364,451.0714286,421.2698413,436.4705882,638.7755102,737.704918,1489.387755,1378.4,794.6875,1405.090909,1307.5,791.9402985,1531.47541,1433.225806,1210,1953.469388,1837.142857,538.9041096,706.1764706,906.7647059,441.8181818,548.3783784,549.7297297,388.1395349,405.1764706,405.1764706],
    [10,2151.428571,2151.428571,10,2151.428571,2151.428571,10,2553.75,2553.75,10,1678.75,1678.75,10,1775.714286,1775.714286,10,775.7142857,775.7142857,10,10,10,10,10,10,969.0909091,1775.714286,1775.714286,1292.222222,1678.75,1678.75,969.0909091,1638.823529,1638.823529,10,2151.428571,2151.428571],
    [2507.5,2507.5,2507.5,2507.5,2507.5,2507.5,1437.142857,1437.142857,1437.142857,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1033.125,1033.125,1033.125,1437.142857,1437.142857,1437.142857,4006,4006,4006,1437.142857,1437.142857,1437.142857],
    [640,936.1904762,936.1904762,640,936.1904762,936.1904762,519.1666667,754.4,706.4,401.5,520.9756098,526.5,342.7906977,387.6595745,459.5918367,572,574.2857143,706.25,600.4761905,571.372549,694.8,644.1025641,633.7777778,777.2727273,597.6470588,811.3043478,1037.391304,833,868.8888889,868.8888889,833,750.5882353,750.5882353,913.6842105,1145.263158,1145.263158],
    [986.4583333,1088.020833,1492.1875,989.0625,1090.625,1492.1875,1175.520833,1285.9375,1441.666667,994.7916667,994.7916667,1154.6875,1194.387755,1194.387755,1376.020408,1137.244898,1596.938776,1762.244898,1195.3125,1673.958333,1720.3125,1254.6875,1747.938144,1710.309278,1268.229167,1753.092784,1777.319588,1147.44898,1147.44898,1298.979592,1120.833333,1224.479167,1598.958333,986.4583333,1088.020833,1492.1875],
    [3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,3394.444444,3394.444444,3394.444444,3394.444444,2568.75,2568.75,3394.444444,2568.75,2568.75,1507.142857,1507.142857,1507.142857,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333]
])


location_names = [
    "River", "Cliff Top", "River Mouth", "Pond", "Sea", "Pier"
]

time_names = [
    "Jan 9a-4p", "Jan 4a-9a/4p-9p", "Jan 9p-4a",
    "Feb 9a-4p", "Feb 4a-9a/4p-9p", "Feb 9p-4a",
    "Mar 9a-4p", "Mar 4a-9a/4p-9p", "Mar 9p-4a",
    "Apr 9a-4p", "Apr 4a-9a/4p-9p", "Apr 9p-4a",
    "May 9a-4p", "May 4a-9a/4p-9p", "May 9p-4a",
    "Jun 9a-4p", "Jun 4a-9a/4p-9p", "Jun 9p-4a",
    "Jul 9a-4p", "Jul 4a-9a/4p-9p", "Jul 9p-4a",
    "Aug 9a-4p", "Aug 4a-9a/4p-9p", "Aug 9p-4a",
    "Sep 9a-4p", "Sep 4a-9a/4p-9p", "Sep 9p-4a",
    "Oct 9a-4p", "Oct 4a-9a/4p-9p", "Oct 9p-4a",
    "Nov 9a-4p", "Nov 4a-9a/4p-9p", "Nov 9p-4a",
    "Dec 9a-4p", "Dec 4a-9a/4p-9p", "Dec 9p-4a",
]

Season = namedtuple('Season', ['name', 'time_indices'])

# Define Northern Hemisphere seasons based on your 36 time index
# Each month has 3 time indices. Index is (month - 1) * 3 + time_period_index (0, 1, or 2)
# Winter (Dec, Jan, Feb): Indices 33-35 (Dec), 0-5 (Jan, Feb)
winter_indices = list(range(33, 36)) + list(range(0, 6))
# Spring (Mar, Apr, May): Indices 6-14
spring_indices = list(range(6, 15))
# Summer (Jun, Jul, Aug): Indices 15-23
summer_indices = list(range(15, 24))
# Fall (Sep, Oct, Nov): Indices 24-32
fall_indices = list(range(24, 33))

seasons = [
    Season("Winter (Dec, Jan, Feb)", winter_indices),
    Season("Spring (Mar, Apr, May)", spring_indices),
    Season("Summer (Jun, Jul, Aug)", summer_indices),
    Season("Fall (Sep, Oct, Nov)", fall_indices)
]

# --- SOLUTION FUNCTION ---

def print_solution(model, x, y, v, time_names, location_names, season_name):
    """Prints the results for a specific optimization run."""
    print(f"\n=======================================================")
    print(f"--- OPTIMAL SOLUTION FOR: {season_name} ---")
    print(f"=======================================================")
    if model.Status == GRB.OPTIMAL:
        print(f"Optimal Expected Profit: {model.ObjVal:,.2f} Bells")

        # Get chosen time period
        chosen_time_index = -1
        for t in range(T):
            if y[t].X > 0.5:
                chosen_time_index = t
                break
        
        if chosen_time_index != -1:
            chosen_time_text = time_names[chosen_time_index]
            print("\nChosen Optimal Month/Time Block:")
            print(f"  **{chosen_time_text}**")
        else:
            print("\nError: No single time block chosen.")


        # Get visited locations
        visited_indices = [l for l in range(L) if v[l].X > 0.5]
        visited_locations = [location_names[i] for i in visited_indices]
        
        # Determine the maximum length for clean formatting
        max_len = max(len(name) for name in visited_locations) if visited_locations else 0

        print("\nOptimal Cast Distribution:")
        if chosen_time_index != -1:
            for l in visited_indices:
                casts = x[l, chosen_time_index].X
                print(f"  {location_names[l]:<{max_len}} : {casts:>5.1f} casts")
        
    elif model.Status == GRB.INFEASIBLE:
        print("Model is Infeasible. Check constraints.")
    else:
        print(f"Optimization Status: {model.Status}")

# --- SEASONAL OPTIMIZATION LOOP ---

for season in seasons:
    # 1. Create a fresh model for each season
    model = gp.Model(f"Fishing_{season.name.replace(' ', '_').replace('-', '')}")
    model.setParam('OutputFlag', 0) # Suppress Gurobi output for cleaner console

    # 2. Add Variables
    x = model.addVars(L, T, lb=0, vtype=GRB.CONTINUOUS, name="x")
    y = model.addVars(T, vtype=GRB.BINARY, name="y")
    v = model.addVars(L, vtype=GRB.BINARY, name="v")
    
    # 3. Add General Constraints
    model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) <= MAX_CASTS, "MaxCasts")
    model.addConstr(gp.quicksum(y[t] for t in range(T)) == 1, "OneTimePeriod")

    # Link x to y (Only cast if time period y[t] is chosen)
    for l in range(L):
        for t in range(T):
            model.addConstr(x[l,t] <= M * y[t], f"LinkXY_l{l}_t{t}")

    # Visit at least 3 locations
    model.addConstr(gp.quicksum(v[l] for l in range(L)) >= 3, "Min3Locations")

    # Minimum casts if visited
    for l in range(L):
        model.addConstr(gp.quicksum(x[l,t] for t in range(T)) >= MIN_VISITS * v[l], f"MinCasts_l{l}")

    # No casts at unvisited locations (Ensures casts only happen at visited locations)
    for l in range(L):
        for t in range(T):
            model.addConstr(x[l,t] <= M * v[l], f"LinkXV_l{l}_t{t}")
            
    # Add constraint for a minimum total number of casts (3 locations * 5 casts/location = 15 casts)
    model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) >= 15, "MinTotalCastsForFeasibility")

    # 4. ADD SEASONAL CONSTRAINT 
    # Enforce y[t] = 0 for all out-of-season periods
    all_time_indices = set(range(T))
    seasonal_indices = set(season.time_indices)
    out_of_season_indices = list(all_time_indices - seasonal_indices)
    
    # We must choose a time period within the allowed seasonal indices.
    model.addConstr(gp.quicksum(y[t] for t in out_of_season_indices) == 0, "SeasonalFilter")

    # 5. Objective: maximize expected profit
    model.setObjective(gp.quicksum(p[l,t]*x[l,t] for l in range(L) for t in range(T)), GRB.MAXIMIZE)

    # 6. Solve and Print
    model.optimize()
    print_solution(model, x, y, v, time_names, location_names, season.name)


--- OPTIMAL SOLUTION FOR: Winter (Dec, Jan, Feb) ---
Optimal Expected Profit: 98,961.31 Bells

Chosen Optimal Month/Time Block:
  **Feb 4a-9a/4p-9p**

Optimal Cast Distribution:
  Cliff Top   :   5.0 casts
  River Mouth :   5.0 casts
  Pier        :  20.0 casts

--- OPTIMAL SOLUTION FOR: Spring (Mar, Apr, May) ---
Optimal Expected Profit: 95,643.75 Bells

Chosen Optimal Month/Time Block:
  **Mar 9p-4a**

Optimal Cast Distribution:
  Cliff Top :   5.0 casts
  Sea       :   5.0 casts
  Pier      :  20.0 casts

--- OPTIMAL SOLUTION FOR: Summer (Jun, Jul, Aug) ---
Optimal Expected Profit: 83,284.14 Bells

Chosen Optimal Month/Time Block:
  **Jul 4a-9a/4p-9p**

Optimal Cast Distribution:
  River :   5.0 casts
  Sea   :   5.0 casts
  Pier  :  20.0 casts

--- OPTIMAL SOLUTION FOR: Fall (Sep, Oct, Nov) ---
Optimal Expected Profit: 107,230.78 Bells

Chosen Optimal Month/Time Block:
  **Nov 9p-4a**

Optimal Cast Distribution:
  Cliff Top   :   5.0 casts
  River Mouth :  20.0 casts
  Pier       

In [29]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from collections import namedtuple

# FORCE BY TIME 

# --- MODEL SETUP ---
L = 6 # 6 fishing locations
T = 36 # 36 time periods (12 months * 3 periods)

# CONSTANTS
MAX_CASTS = 30
MIN_VISITS = 5
M = 30

# Expected profit matrix p[l, t] (Same as before)
p = np.array([
    [380.2352941,402.4390244,402.4390244,380.2352941,402.4390244,402.4390244,369.6296296,387.4285714,387.4285714,312.7272727,280.3636364,451.0714286,421.2698413,436.4705882,638.7755102,737.704918,1489.387755,1378.4,794.6875,1405.090909,1307.5,791.9402985,1531.47541,1433.225806,1210,1953.469388,1837.142857,538.9041096,706.1764706,906.7647059,441.8181818,548.3783784,549.7297297,388.1395349,405.1764706,405.1764706],
    [10,2151.428571,2151.428571,10,2151.428571,2151.428571,10,2553.75,2553.75,10,1678.75,1678.75,10,1775.714286,1775.714286,10,775.7142857,775.7142857,10,10,10,10,10,10,969.0909091,1775.714286,1775.714286,1292.222222,1678.75,1678.75,969.0909091,1638.823529,1638.823529,10,2151.428571,2151.428571],
    [2507.5,2507.5,2507.5,2507.5,2507.5,2507.5,1437.142857,1437.142857,1437.142857,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1033.125,1033.125,1033.125,1437.142857,1437.142857,1437.142857,4006,4006,4006,1437.142857,1437.142857,1437.142857],
    [640,936.1904762,936.1904762,640,936.1904762,936.1904762,519.1666667,754.4,706.4,401.5,520.9756098,526.5,342.7906977,387.6595745,459.5918367,572,574.2857143,706.25,600.4761905,571.372549,694.8,644.1025641,633.7777778,777.2727273,597.6470588,811.3043478,1037.391304,833,868.8888889,868.8888889,833,750.5882353,750.5882353,913.6842105,1145.263158,1145.263158],
    [986.4583333,1088.020833,1492.1875,989.0625,1090.625,1492.1875,1175.520833,1285.9375,1441.666667,994.7916667,994.7916667,1154.6875,1194.387755,1194.387755,1376.020408,1137.244898,1596.938776,1762.244898,1195.3125,1673.958333,1720.3125,1254.6875,1747.938144,1710.309278,1268.229167,1753.092784,1777.319588,1147.44898,1147.44898,1298.979592,1120.833333,1224.479167,1598.958333,986.4583333,1088.020833,1492.1875],
    [3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,3394.444444,3394.444444,3394.444444,3394.444444,2568.75,2568.75,3394.444444,2568.75,2568.75,1507.142857,1507.142857,1507.142857,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333]
])


location_names = [
    "River", "Cliff Top", "River Mouth", "Pond", "Sea", "Pier"
]

time_names = [
    "Jan 9a-4p", "Jan 4a-9a/4p-9p", "Jan 9p-4a",
    "Feb 9a-4p", "Feb 4a-9a/4p-9p", "Feb 9p-4a",
    "Mar 9a-4p", "Mar 4a-9a/4p-9p", "Mar 9p-4a",
    "Apr 9a-4p", "Apr 4a-9a/4p-9p", "Apr 9p-4a",
    "May 9a-4p", "May 4a-9a/4p-9p", "May 9p-4a",
    "Jun 9a-4p", "Jun 4a-9a/4p-9p", "Jun 9p-4a",
    "Jul 9a-4p", "Jul 4a-9a/4p-9p", "Jul 9p-4a",
    "Aug 9a-4p", "Aug 4a-9a/4p-9p", "Aug 9p-4a",
    "Sep 9a-4p", "Sep 4a-9a/4p-9p", "Sep 9p-4a",
    "Oct 9a-4p", "Oct 4a-9a/4p-9p", "Oct 9p-4a",
    "Nov 9a-4p", "Nov 4a-9a/4p-9p", "Nov 9p-4a",
    "Dec 9a-4p", "Dec 4a-9a/4p-9p", "Dec 9p-4a",
]

TimeGroup = namedtuple('TimeGroup', ['name', 'time_indices'])

# Define the three daily time groups:
# Group 0: 9a-4p (indices 0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33) -> t % 3 == 0
# Group 1: 4a-9a/4p-9p (indices 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34) -> t % 3 == 1
# Group 2: 9p-4a (indices 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35) -> t % 3 == 2

time_groups = [
    TimeGroup("Daytime (9 a.m. - 4 p.m.)", [t for t in range(T) if t % 3 == 0]),
    TimeGroup("Dawn/Dusk (4 a.m.-9 a.m. / 4 p.m.-9 p.m.)", [t for t in range(T) if t % 3 == 1]),
    TimeGroup("Nighttime (9 p.m. - 4 a.m.)", [t for t in range(T) if t % 3 == 2])
]

# --- SOLUTION FUNCTION ---

def print_solution(model, x, y, v, time_names, location_names, group_name):
    """Prints the results for a specific optimization run."""
    print(f"\n=======================================================")
    print(f"--- OPTIMAL SOLUTION FOR: {group_name} ---")
    print(f"=======================================================")
    if model.Status == GRB.OPTIMAL:
        print(f"Optimal Expected Profit: {model.ObjVal:,.2f} Bells")

        # Get chosen time period
        chosen_time_index = -1
        for t in range(T):
            if y[t].X > 0.5:
                chosen_time_index = t
                break
        
        if chosen_time_index != -1:
            chosen_time_text = time_names[chosen_time_index]
            print("\nChosen Optimal Month/Time Block:")
            print(f"  **{chosen_time_text}**")
        else:
            print("\nError: No single time block chosen.")

        # Get visited locations
        visited_indices = [l for l in range(L) if v[l].X > 0.5]
        visited_locations = [location_names[i] for i in visited_indices]
        
        max_len = max(len(name) for name in visited_locations) if visited_locations else 0

        print("\nOptimal Cast Distribution:")
        if chosen_time_index != -1:
            for l in visited_indices:
                casts = x[l, chosen_time_index].X
                print(f"  {location_names[l]:<{max_len}} : {casts:>5.1f} casts")
        
    elif model.Status == GRB.INFEASIBLE:
        print("Model is Infeasible. Check constraints.")
    else:
        print(f"Optimization Status: {model.Status}")

# --- TIME GROUP OPTIMIZATION LOOP ---

for group in time_groups:
    # 1. Create a fresh model for each time group
    model = gp.Model(f"Fishing_{group.name.replace(' ', '_').replace('/', '')}")
    model.setParam('OutputFlag', 0) # Suppress Gurobi output

    # 2. Add Variables
    x = model.addVars(L, T, lb=0, vtype=GRB.CONTINUOUS, name="x")
    y = model.addVars(T, vtype=GRB.BINARY, name="y")
    v = model.addVars(L, vtype=GRB.BINARY, name="v")
    
    # 3. Add General Constraints (Max 30 casts, Min 3 locations, etc.)
    model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) <= MAX_CASTS, "MaxCasts")
    model.addConstr(gp.quicksum(y[t] for t in range(T)) == 1, "OneTimePeriod")

    for l in range(L):
        for t in range(T):
            model.addConstr(x[l,t] <= M * y[t], f"LinkXY_l{l}_t{t}")

    model.addConstr(gp.quicksum(v[l] for l in range(L)) >= 3, "Min3Locations")

    for l in range(L):
        model.addConstr(gp.quicksum(x[l,t] for t in range(T)) >= MIN_VISITS * v[l], f"MinCasts_l{l}")

    for l in range(L):
        for t in range(T):
            model.addConstr(x[l,t] <= M * v[l], f"LinkXV_l{l}_t{t}")
            
    # Add constraint for minimum total casts required to meet the 5-cast minimum in 3 locations
    model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) >= 15, "MinTotalCastsForFeasibility")

    # 4. ADD TIME GROUP CONSTRAINT (The key change)
    # The chosen time period y[t] must be one of the indices in this specific daily time group.
    
    # 4.1 Create a list of time indices *not* in the current time group
    all_time_indices = set(range(T))
    group_indices = set(group.time_indices)
    out_of_group_indices = list(all_time_indices - group_indices)
    
    # 4.2 Enforce y[t] = 0 for all out-of-group periods
    # This forces the solution to pick one of the 12 month options within the current daily time slot.
    model.addConstr(gp.quicksum(y[t] for t in out_of_group_indices) == 0, "TimeGroupFilter")

    # 5. Objective: maximize expected profit
    model.setObjective(gp.quicksum(p[l,t]*x[l,t] for l in range(L) for t in range(T)), GRB.MAXIMIZE)

    # 6. Solve and Print
    model.optimize()
    print_solution(model, x, y, v, time_names, location_names, group.name)


--- OPTIMAL SOLUTION FOR: Daytime (9 a.m. - 4 p.m.) ---
Optimal Expected Profit: 104,640.83 Bells

Chosen Optimal Month/Time Block:
  **Nov 9a-4p**

Optimal Cast Distribution:
  River Mouth :  20.0 casts
  Sea         :   5.0 casts
  Pier        :   5.0 casts

--- OPTIMAL SOLUTION FOR: Dawn/Dusk (4 a.m.-9 a.m. / 4 p.m.-9 p.m.) ---
Optimal Expected Profit: 107,230.78 Bells

Chosen Optimal Month/Time Block:
  **Nov 4a-9a/4p-9p**

Optimal Cast Distribution:
  Cliff Top   :   5.0 casts
  River Mouth :  20.0 casts
  Pier        :   5.0 casts

--- OPTIMAL SOLUTION FOR: Nighttime (9 p.m. - 4 a.m.) ---
Optimal Expected Profit: 107,230.78 Bells

Chosen Optimal Month/Time Block:
  **Nov 9p-4a**

Optimal Cast Distribution:
  Cliff Top   :   5.0 casts
  River Mouth :  20.0 casts
  Pier        :   5.0 casts


In [34]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
L = 6
T = 36
MAX_CASTS = 30
M = 30
MIN_VISITS = 5

# TIME 
# Expected fish prices (your p matrix)
p = np.array([
    [380.2352941,402.4390244,402.4390244,380.2352941,402.4390244,402.4390244,369.6296296,387.4285714,387.4285714,312.7272727,280.3636364,451.0714286,421.2698413,436.4705882,638.7755102,737.704918,1489.387755,1378.4,794.6875,1405.090909,1307.5,791.9402985,1531.47541,1433.225806,1210,1953.469388,1837.142857,538.9041096,706.1764706,906.7647059,441.8181818,548.3783784,549.7297297,388.1395349,405.1764706,405.1764706],
    [10,2151.428571,2151.428571,10,2151.428571,2151.428571,10,2553.75,2553.75,10,1678.75,1678.75,10,1775.714286,1775.714286,10,775.7142857,775.7142857,10,10,10,10,10,10,969.0909091,1775.714286,1775.714286,1292.222222,1678.75,1678.75,969.0909091,1638.823529,1638.823529,10,2151.428571,2151.428571],
    [2507.5,2507.5,2507.5,2507.5,2507.5,2507.5,1437.142857,1437.142857,1437.142857,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1033.125,1033.125,1033.125,1437.142857,1437.142857,1437.142857,4006,4006,4006,1437.142857,1437.142857,1437.142857],
    [640,936.1904762,936.1904762,640,936.1904762,936.1904762,519.1666667,754.4,706.4,401.5,520.9756098,526.5,342.7906977,387.6595745,459.5918367,572,574.2857143,706.25,600.4761905,571.372549,694.8,644.1025641,633.7777778,777.2727273,597.6470588,811.3043478,1037.391304,833,868.8888889,868.8888889,833,750.5882353,750.5882353,913.6842105,1145.263158,1145.263158],
    [986.4583333,1088.020833,1492.1875,989.0625,1090.625,1492.1875,1175.520833,1285.9375,1441.666667,994.7916667,994.7916667,1154.6875,1194.387755,1194.387755,1376.020408,1137.244898,1596.938776,1762.244898,1195.3125,1673.958333,1720.3125,1254.6875,1747.938144,1710.309278,1268.229167,1753.092784,1777.319588,1147.44898,1147.44898,1298.979592,1120.833333,1224.479167,1598.958333,986.4583333,1088.020833,1492.1875],
    [3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,1507.142857,3394.444444,3394.444444,3394.444444,3394.444444,2568.75,2568.75,3394.444444,2568.75,2568.75,1507.142857,1507.142857,1507.142857,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333,3783.333333]
])


location_names = ["River", "Cliff Top", "River Mouth", "Pond", "Sea", "Pier"]

time_names = [
    "Jan 9a-4p", "Jan 4a-9a/4p-9p", "Jan 9p-4a",
    "Feb 9a-4p", "Feb 4a-9a/4p-9p", "Feb 9p-4a",
    "Mar 9a-4p", "Mar 4a-9a/4p-9p", "Mar 9p-4a",
    "Apr 9a-4p", "Apr 4a-9a/4p-9p", "Apr 9p-4a",
    "May 9a-4p", "May 4a-9a/4p-9p", "May 9p-4a",
    "Jun 9a-4p", "Jun 4a-9a/4p-9p", "Jun 9p-4a",
    "Jul 9a-4p", "Jul 4a-9a/4p-9p", "Jul 9p-4a",
    "Aug 9a-4p", "Aug 4a-9a/4p-9p", "Aug 9p-4a",
    "Sep 9a-4p", "Sep 4a-9a/4p-9p", "Sep 9p-4a",
    "Oct 9a-4p", "Oct 4a-9a/4p-9p", "Oct 9p-4a",
    "Nov 9a-4p", "Nov 4a-9a/4p-9p", "Nov 9p-4a",
    "Dec 9a-4p", "Dec 4a-9a/4p-9p", "Dec 9p-4a",
]
model = gp.Model("Fishing")
model.setParam('OutputFlag', 0)

x = model.addVars(L, T, lb=0, vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(T, vtype=GRB.BINARY, name="y")
v = model.addVars(L, vtype=GRB.BINARY, name="v")

# Total 30 casts
model.addConstr(gp.quicksum(x[l,t] for l in range(L) for t in range(T)) <= MAX_CASTS)

# Exactly one time period
model.addConstr(gp.quicksum(y[t] for t in range(T)) == 1)

# Cast only in chosen time
for l in range(L):
    for t in range(T):
        model.addConstr(x[l,t] <= M * y[t])

# At least 3 locations
model.addConstr(gp.quicksum(v[l] for l in range(L)) >= 3)

# If visit, must cast ≥5
for l in range(L):
    model.addConstr(gp.quicksum(x[l,t] for t in range(T)) >= MIN_VISITS * v[l])

# No casts if not visited
for l in range(L):
    for t in range(T):
        model.addConstr(x[l,t] <= M * v[l])

# Objective
model.setObjective(gp.quicksum(p[l,t] * x[l,t] for l in range(L) for t in range(T)),
                   GRB.MAXIMIZE)

model.optimize()

def print_solution(model, x, y, v, time_names, location_names):
    print("Optimal Expected Profit:", model.ObjVal)

    chosen_t = max(range(T), key=lambda t: y[t].X)
    print("\nChosen Time Period:", time_names[chosen_t])

    print("\nVisited Locations:")
    for l in range(L):
        if v[l].X > 0.5:
            print(f"  {location_names[l]}: {x[l,chosen_t].X:.0f} casts")

print_solution(model, x, y, v, time_names, location_names)

def compute_location_shadow_values(model, v, x, y, L):
    baseline = model.ObjVal
    out = []

    for l in range(L):
        # Force V=1
        m1 = model.copy()
        v1 = m1.getVarByName(f"v[{l}]")
        v1.lb = 1; v1.ub = 1
        m1.optimize()
        obj1 = m1.ObjVal if m1.Status == GRB.OPTIMAL else None

        # Force V=0
        m2 = model.copy()
        v2 = m2.getVarByName(f"v[{l}]")
        v2.lb = 0; v2.ub = 0
        m2.optimize()
        obj0 = m2.ObjVal if m2.Status == GRB.OPTIMAL else None

        out.append({
            "Location": l,
            "Baseline": baseline,
            "Obj(V=1)": obj1,
            "Obj(V=0)": obj0,
            "Gain_if_Forced_1": None if obj1 is None else obj1 - baseline,
            "Loss_if_Forbidden_0": None if obj0 is None else baseline - obj0
        })
    return out

def compute_time_period_shadow_values(model, y, T):
    baseline = model.ObjVal
    out = []

    for t in range(T):
        m = model.copy()
        yt = m.getVarByName(f"y[{t}]")
        yt.lb = 1; yt.ub = 1
        m.optimize()
        obj = m.ObjVal if m.Status == GRB.OPTIMAL else None
        delta = None if obj is None else obj - baseline

        out.append({
            "Time": t,
            "Baseline": baseline,
            "Obj_if_forced": obj,
            "Delta": delta
        })
    return out


loc_shadow = compute_location_shadow_values(model, v, x, y, L)
time_shadow = compute_time_period_shadow_values(model, y, T)
df_loc = pd.DataFrame(loc_shadow)

df_loc = df_loc.rename(columns={
    "Location": "Loc Index",
    "Obj(V=1)": "Obj if V=1",
    "Obj(V=0)": "Obj if V=0",
    "Gain_if_Forced_1": "Reduced Cost (Gain if Forced)",
    "Loss_if_Forbidden_0": "Shadow Price (Loss if Forbidden)"
})

df_loc["Location"] = df_loc["Loc Index"].apply(lambda i: location_names[i])
df_loc = df_loc[[
    "Location",
    "Baseline",
    "Obj if V=1",
    "Obj if V=0",
    "Reduced Cost (Gain if Forced)",
    "Shadow Price (Loss if Forbidden)"
]]
df_loc = df_loc.round(2)

print("\n==================== LOCATION SENSITIVITY TABLE ====================")
print(df_loc.to_string(index=False))
df_time = pd.DataFrame(time_shadow)

df_time = df_time.rename(columns={
    "Time": "Time Index",
    "Obj_if_forced": "Obj if Forced",
    "Delta": "Profit Change"
})

df_time["Time Period"] = df_time["Time Index"].apply(lambda i: time_names[i])
df_time = df_time[[
    "Time Period",
    "Baseline",
    "Obj if Forced",
    "Profit Change"
]]
df_time = df_time.round(2)

print("\n==================== TIME PERIOD SENSITIVITY TABLE ====================")
print(df_time.to_string(index=False))

Optimal Expected Profit: 107230.78431

Chosen Time Period: Nov 4a-9a/4p-9p

Visited Locations:
  Cliff Top: 5 casts
  River Mouth: 20 casts
  Pier: 5 casts

==================== LOCATION SENSITIVITY TABLE ====================
   Location  Baseline  Obj if V=1  Obj if V=0  Reduced Cost (Gain if Forced)  Shadow Price (Loss if Forbidden)
      River 107230.78   101785.32   107230.78                       -5445.47                              0.00
  Cliff Top 107230.78   107230.78   107031.46                           0.00                            199.33
River Mouth 107230.78   107230.78    95643.75                           0.00                          11587.03
       Pond 107230.78   103201.67   107230.78                       -4029.12                              0.00
        Sea 107230.78   107031.46   107230.78                        -199.33                              0.00
       Pier 107230.78   107230.78    96308.91                           0.00                          10921.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 34.6 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]
